In [8]:
!which python


/home/calcgen2/anaconda3/bin/python


In [9]:
!python --version

Python 3.7.3


In [10]:
cfg_file = 'cfg/yolov3.cfg'

In [11]:
!ls cfg

yolov3.cfg


In [12]:
# read the file and split each line
with open(cfg_file, 'r') as file:
    lines = file.read().splitlines()


In [13]:
lines = [line for line in lines if len(line)>0]
lines = [line for line in lines if line[0] != '#']
lines = [x.rstrip().lstrip() for x in lines]           # get rid of fringe whitespaces

In [14]:
for line in lines:
    if line[0] == "[":               # This marks the start of a new block
        if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
            blocks.append(block)     # add it the blocks list
            block = {}               # re-init the block
        block["type"] = line[1:-1].rstrip()     
    else:
        key,value = line.split("=") 
        block[key.rstrip()] = value.lstrip()
blocks.append(block)

NameError: name 'block' is not defined

In [15]:
blocks = []
block = {}
for line in lines:
    if line[0] == '[':
        if block:
            blocks.append(block)
            block = {}
        layer_name = line.strip('[]').rstrip().lstrip()
        # get the name of the layer
        block['name'] = layer_name
    else:
        key, value = line.split('=')
        key = key.rstrip().lstrip()
        value = value.rstrip().lstrip()
        values = value.split(',  ')
        # check if it is a 2d attribute
        if len(values) > 1:
            values = [value.split(',') for value in values]
        else:
            values = value.split(',')
        # check if it is a single element
        if len(values) == 1:
            values = values[0]
        block[key] = values
blocks.append(block)

In [16]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np


In [17]:
class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()


In [18]:
class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors


In [19]:
def create_modules(blocks):
    net_info = blocks[0]     #Captures the information about the input and pre-processing    
    module_list = nn.ModuleList()
    prev_filters = 3
    output_filters = []
    for index, x in enumerate(blocks[1:]):
        module = nn.Sequential()

        #check the name of block
        #create a new module for the block
        #append to module_list
        if (x["name"] == "convolutional"):
            #Get the info about the layer
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False
            except:
                batch_normalize = 0
                bias = True

            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])

            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0

            #Add the convolutional layer
            conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias = bias)
            module.add_module("conv_{0}".format(index), conv)

            #Add the Batch Norm Layer
            if batch_normalize:
                bn = nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)

            #Check the activation. 
            #It is either Linear or a Leaky ReLU for YOLO
            if activation == "leaky":
                activn = nn.LeakyReLU(0.1, inplace = True)
                module.add_module("leaky_{0}".format(index), activn)

        #If it's an upsampling layer
        #We use Bilinear2dUpsampling
        elif (x["name"] == "upsample"):
            stride = int(x["stride"])
            upsample = nn.Upsample(scale_factor = 2, mode = "bilinear")
            module.add_module("upsample_{}".format(index), upsample)
        #If it is a route layer
        elif (x["name"] == "route"):
#             x["layers"] = x["layers"].split(',')
            #Start  of a route
#             start = int(x["layers"][0])
            #end, if there exists one.
            print('route value: ', x["layers"])

            try:
                start = int(x["layers"][0])
                end = int(x["layers"][1])
            except:
                start = int(x["layers"])
                end = 0
            #Positive anotation
            if start > 0: 
                start = start - index
            if end > 0:
                end = end - index
            route = EmptyLayer()
            module.add_module("route_{0}".format(index), route)
            if end < 0:
                print('two add')
                filters = output_filters[index + start] + output_filters[index + end]
            else:
                print('one add', start)
                filters= output_filters[index + start]
            print('filters: ', filters)
            print('***********')

        #shortcut corresponds to skip connection
        elif x["name"] == "shortcut":
            shortcut = EmptyLayer()
            module.add_module("shortcut_{}".format(index), shortcut)
        #Yolo is the detection layer
        elif x["name"] == "yolo":
            mask = x["mask"]
            mask = [int(x) for x in mask]

            anchors = x["anchors"]
            anchors = [anchors[i] for i in mask]

            detection = DetectionLayer(anchors)
            module.add_module("Detection_{}".format(index), detection)
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
    return (net_info, module_list)

In [20]:
create_modules(blocks)

route value:  -4
one add -4
filters:  512
***********
route value:  ['-1', ' 61']
two add
filters:  768
***********
route value:  -4
one add -4
filters:  256
***********
route value:  ['-1', ' 36']
two add
filters:  384
***********


({'name': 'net',
  'batch': '64',
  'subdivisions': '16',
  'width': '608',
  'height': '608',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': ['400000', '450000'],
  'scales': ['.1', '.1']},
 ModuleList(
   (0): Sequential(
     (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
   )
   (1): Sequential(
     (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
   )
   (2): Sequential(
     (conv_2): Conv2d(64, 32, ke

In [6]:
# my function
def create_modules(blocks):
    print('my function')
    # create the actual nn using the blocks
    # Captures the information about the input and pre-processing
    net_info = blocks[0]
    module_list = nn.ModuleList()
    prev_filters = 3
    output_filters = []

# iterate throught the blocks and create module for each block
    for index, layer in enumerate(blocks[1:]):
        module = nn.Sequential()
        # check the type of block
        # create a new moduel for the block
        # append to module_list
        if (layer['name'] == 'convolutional'):
            try:
                batch_norm = int(layer['batch_normalize'])
                bias = False
            except:
                batch_norm = 0
                bias = True
            filters = int(layer['filters'])
            kernel_size = int(layer['size'])
            stride = int(layer['stride'])
            padding = int(layer['pad'])

            activation = layer['activation']

            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0

            # Add the convolutional layer
            conv_layer = nn.Conv2d(
                in_channels=prev_filters,
                out_channels=filters,
                kernel_size=kernel_size,
                stride=stride,
                padding=pad,
                bias=bias
            )
            module.add_module('Conv_{}'.format(index), conv_layer)

            # Add if there is batch norm
            if batch_norm:
                batchN_layer = nn.BatchNorm2d(
                    num_features=filters
                )
                module.add_module('BatchN_{}'.format(index), batchN_layer)

            # Check the type of activation and add
            if activation == 'leaky':
                # here the negative slope is 0.01 which is default
                # but the tutorial uses 0.1 why?????
                l_relu_layer = nn.LeakyReLU(
                    inplace=True,
                    negative_slope=0.01
                )
                module.add_module('LRelu_{}'.format(index), l_relu_layer)

        elif layer['name'] == 'upsample':
            stride = int(layer['stride'])
            upsample_layer = nn.Upsample(
                scale_factor=stride,
                mode='bilinear'
            )
            module.add_module('Upsample_{}'.format(index), upsample_layer)

        elif layer['name'] == 'route':
            prev_layers = layer['layers']
            print('route value: ', prev_layers)
            # get the prev layers output
            # concatenate them and send as output
            try:
                # route layers may have two / one layers
                first_layer = int(prev_layers[0])
                second_layer = int(prev_layers[1])
            except:
                first_layer = int(prev_layers)
                second_layer = 0

            # calculate the number of outputs
            if not second_layer:
                print('one add', first_layer)
                filters = output_filters[index + first_layer]
            else:
                print('two add', second_layer)
                
                filters = output_filters[index + first_layer] +\
                    output_filters[second_layer]
            print('filters: ', filters)
            print('******')
            route_layer = EmptyLayer()
            module.add_module('Route_{}'.format(index), route_layer)
        elif layer['name'] == 'shortcut':
            # shortcut = Emptylayer()
            prev_layers = layer['from']
            activation = layer['activation']

            # filters = output_filters[prev_layers]
            shorcut_layer = EmptyLayer()
            module.add_module('Shorcut_{}'.format(index), shorcut_layer)
        elif layer['name'] == 'yolo':
            masks = layer['mask']
            masks = [int(mask) for mask in masks]

            anchors = layer['anchors']
            detection_layer = DetectionLayer(anchors)
            module.add_module("Detection_{}".format(index),
                              detection_layer)

        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
    return (net_info, module_list)

In [7]:
create_modules(blocks)

NameError: name 'blocks' is not defined

In [42]:
import numpy as np


In [44]:
list(np.meshgrid(5,5))

[array([[5]]), array([[5]])]

In [45]:
a =np.meshgrid(5,5)

In [49]:
a[1,1]

TypeError: list indices must be integers or slices, not tuple

In [51]:
x = np.arange(5)

In [52]:
x

array([0, 1, 2, 3, 4])

In [64]:
a, b = np.meshgrid(x,x)

In [63]:
torch.__version__

'1.0.0'

In [65]:
a.shape

(5, 5)

In [69]:
x_offset = torch.FloatTensor(a).view(-1,1)
y_offset = torch.FloatTensor(b).view(-1,1)


In [80]:
torch.cat((x_offset, y_offset), 1).repeat(1,3).shape

torch.Size([25, 6])

In [81]:
x= torch.Tensor((1,2,3))

In [92]:
x.repeat(2*2,1).shape

torch.Size([4, 3])

In [93]:
x.repeat(2*2,1).unsqueeze(0).shape

torch.Size([1, 4, 3])

In [21]:
x =list(range(10))

In [25]:
print(list(enumerate(x)))

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]


In [26]:
608//26


23

In [27]:
from __future__ import division


In [28]:
608//26

23

In [30]:
608/26

23.384615384615383

In [29]:
608//52

11

In [31]:
608/52

11.692307692307692